## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [57]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd

In [42]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io/legacy'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    #print( j['status'] )
    results = {}
    dictionary = {}
    for child in j['children']:
        #print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results'
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [33]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_devars(query)
        
                dict_workflows[filename] = kcresult

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/EGFR_advanced.json
EGFR_advanced
https://arax.ncats.io/?source=ARS&id=0ea9c751-1022-4853-9d7c-0e96d6c39181
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=3a79d34f-47bd-41fd-88ae-0df54b8fa014
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json
A.1_RHOBTB2
https://arax.ncats.io/?source=ARS&id=79b848ae-4cae-4112-a49e-cd809150a690
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi.json
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
https://arax.ncats.io/?source=ARS&id=601e5bfa-c701-4029-b78a-d17a4f0564b0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi.json
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
h

In [55]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_devars_results(val)
    
    workflow_result_messages[keys] = result_status

EGFR_advanced 0ea9c751-1022-4853-9d7c-0e96d6c39181
ara-aragorn Error 0
ara-arax Error 0
ara-bte Error 0
ara-unsecret Done 0
kp-genetics Done 0
kp-molecular Done 0
ara-explanatory Done 0
ara-improving ARS Error 0
kp-cam Done 0
kp-textmining Error 0
kp-openpredict Error 0
kp-cohd Error 0
kp-chp Done 0
kp-icees Error 0
kp-icees-dili Done 0
A.8_EGFR_simple 3a79d34f-47bd-41fd-88ae-0df54b8fa014
ara-aragorn Error 0
ara-arax Done 453
ara-bte Error 0
ara-unsecret Done 63
kp-genetics Done 0
kp-molecular Done 0
ara-explanatory Done 0
ara-improving ARS Error 0
kp-cam Done 0
kp-textmining Error 0
kp-openpredict Error 0
kp-cohd Error 0
kp-chp Done 0
kp-icees Error 0
kp-icees-dili Done 0
A.1_RHOBTB2 79b848ae-4cae-4112-a49e-cd809150a690
ara-aragorn Error 0
ara-arax Done 2
ara-bte Error 0
ara-unsecret Done 0
kp-genetics Done 0
kp-molecular Done 0
ara-explanatory Done 0
ara-improving Done 4
kp-cam Done 0
kp-textmining Error 0
kp-openpredict Error 0
kp-cohd Error 0
kp-chp Done 0
kp-icees Error 0
kp-icees

In [54]:
workflow_result_messages[keys]

{'ara-aragorn': 'Error',
 'ara-arax': 'Results',
 'ara-bte': 'Error',
 'ara-unsecret': 'Error',
 'kp-genetics': 'No Results',
 'kp-molecular': 'Results',
 'ara-explanatory': 'Results',
 'ara-improving': 'Results',
 'kp-cam': 'No Results',
 'kp-textmining': 'Error',
 'kp-openpredict': 'Error',
 'kp-cohd': 'Results',
 'kp-chp': 'No Results',
 'kp-icees': 'Error',
 'kp-icees-dili': 'No Results'}

In [58]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
            final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [59]:
df

,A.1_RHOBTB2,A.8_EGFR_simple,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin,EGFR_advanced
ara-aragorn,Error,Error,Error,Error,Error,Error,Error,No Results,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
ara-arax,Results,Results,Error,Error,No Results,No Results,Error,Error,Results,Results,Error,Error,Results,Results,Error,Error,Error,Error
ara-bte,Error,Error,Error,Error,No Results,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
ara-unsecret,No Results,Results,No Results,No Results,No Results,Error,No Results,Results,No Results,Error,Error,Error,Error,Results,No Results,Results,Results,No Results
kp-genetics,No Results,No Results,Error,Error,Error,Error,Error,No Results,No Results,No Results,Error,Error,Error,No Results,Error,Error,Error,No Results
kp-molecular,No Results,No Results,Error,Error,Error,Error,Error,Results,No Results,Results,Error,Error,Error,No Results,Error,Error,Error,No Results
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,Results,Error,Results,Results,Results,Results,Results,Results,ARS Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Results,No Results,No Results,Error,Error,No Results,Error,Error,No Results,No Results
kp-textmining,Error,Error,Error,Error,No Results,No Results,No Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error


In [60]:
df.to_excel('ara_worklow_progress_tracker_Aug5.xlsx', sheet_name = 'Progress_Tracker')

In [61]:
dict_workflows

{'EGFR_advanced': '0ea9c751-1022-4853-9d7c-0e96d6c39181',
 'A.8_EGFR_simple': '3a79d34f-47bd-41fd-88ae-0df54b8fa014',
 'A.1_RHOBTB2': '79b848ae-4cae-4112-a49e-cd809150a690',
 'B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi': '601e5bfa-c701-4029-b78a-d17a4f0564b0',
 'B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': 'eb26b50a-ce13-49af-9bec-baa59b359ff0',
 'B.4_one-hop-gene-biological-process-or-activity_trapi': 'a1515ff6-d678-45ef-af2c-aa455e27a4c7',
 'B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': '5aacb0d2-117a-4787-a431-050a89eb0e2e',
 'B.3_DILI-one-hop-from-genes': 'bec4bb3d-5452-4e06-9d96-fe86bb0dd803',
 'B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': 'bc04a4b7-9063-45d9-b20f-1fb12512fade',
 'B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': '41c9e98f-7cf6-4cf6-ab46-29872b4dba6e',
 'D.4_tryptophan-kynurenine-with-curie': '21ab398a-ff0b-4194-b761-9c7afa5e7dce',
 'D.1_parkinsons-crohns': '2ca7cef4-398f-4a17-9b0c

In [62]:
pwd

'/Users/priyash/Documents/GitHub/minihackathons/Notebooks'

In [64]:
with open('ara_worklow_IDs_Aug5.txt', 'w') as f:
    print(dict_workflows, file=f)